Imports and User parameters


In [77]:
import pandas as pd
import numpy as np

df = pd.read_excel("../data/dados-2025.xlsx", sheet_name="Planilha1")
row = df.loc[df["Nome"].str.contains("Arthur Rabello Oliveira")].iloc[0]

iD, m, c, f, N, M = (
    int(row["ID"]),
    float(row["m"]),
    float(row["c"]),
    float(row["f"]),
    float(row["N"]),
    float(row["M"]),
)

print(f"id = {iD}, m = {m}, c = {c}, f = {f}, N = {N}, M = {M}")


id = 2, m = 5.5, c = 0.02, f = 0.35, N = 425.0, M = 3000.0


Problema 1 a)

In [78]:
# ------------------------
# Exercise parameters
# ------------------------
arrival_rate      = 1 / 10  # arrival rate (1 customer every 10 min ⇒ λ = 0.1 min⁻¹)
n_customers = 10_000        # simulation size

fast_fraction = 0.35        # fraction of “fast” customers
slope = 0.02                # slope of the linear pdf (2–6 min)
mean_exp = 5.5              # mean (min) of the exponential service time for the others

# ------------------------
# Utility function: samples service times for fast customers
# pdf:  f(x) = 0.25 + slope·(x – 4)  for  2 < x < 6
# ------------------------
def sample_linear_en(size, slope=slope):
    a = 0.25 - 4 * slope         # f(x) = a + b·x  (b = slope)
    b = slope
    A = b / 2                    # coefficient of x² in the CDF
    B = a
    u = np.random.rand(size)     # uniform samples (0, 1)

    # Solve A·x² + B·x – (2a + 2b) – u = 0  (positive root → x ∈ (2, 6))
    C = -2 * a - 2 * b - u
    x = (-B + np.sqrt(B ** 2 - 4 * A * C)) / (2 * A)
    return x

# ------------------------
# 1. Generate arrivals
# ------------------------
rng = np.random.default_rng()
inter_arrival_times = rng.exponential(scale=1 / arrival_rate, size=n_customers)  # scale = 10 min
arrivals = np.cumsum(inter_arrival_times)

# ------------------------
# 2. Generate service times
# ------------------------
is_fast = rng.random(n_customers) < fast_fraction

service_times = np.empty(n_customers)
service_times[is_fast]  = sample_linear_en(is_fast.sum())
service_times[~is_fast] = rng.exponential(scale=mean_exp, size=(~is_fast).sum())

# ------------------------
# 3. Simulate the queue (M/G/1) and measure waits/idleness
# ------------------------
start_times   = np.empty(n_customers)
end_times     = np.empty(n_customers)
idle_time = 0.0

# customer 0
start_times[0] = arrivals[0]
end_times[0]   = start_times[0] + service_times[0]

# remaining customers
for i in range(1, n_customers):
    if arrivals[i] > end_times[i - 1]:
        idle_time += arrivals[i] - end_times[i - 1]          # server was idle
    start_times[i] = max(arrivals[i], end_times[i - 1])
    end_times[i]   = start_times[i] + service_times[i]

wait_times = start_times - arrivals

# ------------------------
# 4. Requested statistics
# ------------------------
mean_service  = service_times.mean()
std_service   = service_times.std(ddof=1)
mean_wait     = wait_times.mean()
std_wait      = wait_times.std(ddof=1)
p_wait3       = np.mean(wait_times > 3)            # P(wait > 3 min)
idle_fraction = idle_time / end_times[-1]

print(f"Service duration:  mean = {mean_service:.3f} min,  std = {std_service:.3f} min")
print(f"Waiting time:      mean = {mean_wait:.3f} min,  std = {std_wait:.3f} min")
print(f"P(wait > 3 min) = {p_wait3:.3%}")
print(f"Fraction of server idle time = {idle_fraction:.3%}")


Service duration:  mean = 5.060 min,  std = 4.508 min
Waiting time:      mean = 4.522 min,  std = 7.394 min
P(wait > 3 min) = 37.390%
Fraction of server idle time = 49.079%
